In [1]:
bw_test_non_linear_methods_module_path = "data/bw_test_non_linear_methods_module.py"

config_data = {
    "adapters": [
        {
            "adapter_name": "brightway-adapter",
            # "module_path": "...",
            "config": {
                # mandatory
                "bw_project": "ecoinvent_391",
                # optional: default: 1
                "use_k_bw_distributions": 2,
                "store_raw_results": True,
                "store_lca_object": True,
                # config for regionalization
                "simple_regionalization": {
                    # active (default: False)
                    "run_regionalization": False,
                    # select regions to include in the results
                    "select_regions": ["CAT"],
                    # set activities locations as included in this dict
                    "set_node_regions": {"<code>": ["ES", "CAT"]},
                    # Set all activities 'enb_location' that are not in the 'hierarchy' nor 'set_node_regions'
                    "clear_all_other_node_regions": False,
                },
                # config for nonlinear characterization
                "nonlinear_characterization": {
                    # just methods...
                    "methods": {
                        # name as defined in 'methods' in the adapter definition (see below)
                        "GWP1000": {
                            # load method data and create initial linear methods.
                            # will be overwritten by functions defined here
                            "get_defaults_from_original": True,
                            "functions": {
                                # (database, code): <function-name> or anonymous function (e.g. lambda v: v * 1)
                            },
                            # instead of functions field we can also include module_path_function_name a 2 string tuple,
                            # where the first string is a absolute module path and the second a function name
                            # that function will be called and should return the same as functions, which is
                            # a dict of (database, code) tuples and functions (float -> float)
                            # "module_path_function_name": (
                            #     bw_test_non_linear_methods_module_path,
                            #     "wpg_1000"
                            # )
                        }
                    }
                },
            },
            # the methods we want to incoperate (a dict: arbitrary name (used in results) -> bw identifier tuple
            "methods": {
                "GWP1000": [
                    "ReCiPe 2016 v1.03, midpoint (E)",
                    "climate change",
                    "global warming potential (GWP1000)",
                ]
            },
        }
    ],
    # each node in the hierarchy must contain the fields 'name' and 'aggregator' (non-leaf nodes) or 'adapter'
    # the config is passed to the corresponding aggregator/adapter
    # the config for 'single_activity' is used to find the activity and to set a default output
    "hierarchy": {
        "name": "root",
        "aggregator": "sum",
        "children": [
            {
                "name": "single_activity",
                "adapter": "bw",
                "config": {
                    "name": "heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014",
                    "unit": "kilowatt hour",
                    # "code": 'b9d74efa4fd670b1977a3471ec010737',
                    "location": "DK",
                    "default_output": {"unit": "kWh", "magnitude": 1},
                },
            }
        ],
    },
    "config": {"run_adapters_concurrently": False},
}


In [2]:
from enbios.base.experiment import Experiment

exp = Experiment(config_data)
exp.run()

In [7]:
from enbios.bw2.MultiLCA_util import BaseStackedMultiLCA
from enbios.bw2.brightway_experiment_adapter import BrightwayAdapter

bw_adapter: BrightwayAdapter = exp.get_adapter_by_name("brightway-adapter")
# bw_adapter.lca_objects
print(bw_adapter.raw_results)

multi_lca: list[BaseStackedMultiLCA] = bw_adapter.lca_objects["default scenario"]

In [8]:
multi_lca[0].results

In [9]:
multi_lca